In [8]:
# Step 1: Imports
import pandas as pd
import numpy as np




In [9]:
#Step 2: Load Dataset
df = pd.read_csv("../data/Kangaroo.csv")

In [10]:

# Step 3: Initial Exploration
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.isnull().sum().sort_values(ascending=False))
print(df.describe())



(80368, 53)
Index(['Unnamed: 0', 'id', 'url', 'type', 'subtype', 'bedroomCount',
       'bathroomCount', 'province', 'locality', 'postCode', 'habitableSurface',
       'roomCount', 'monthlyCost', 'hasAttic', 'hasBasement',
       'hasDressingRoom', 'diningRoomSurface', 'hasDiningRoom',
       'buildingCondition', 'buildingConstructionYear', 'facedeCount',
       'floorCount', 'streetFacadeWidth', 'hasLift', 'floodZoneType',
       'heatingType', 'hasHeatPump', 'hasPhotovoltaicPanels',
       'hasThermicPanels', 'kitchenSurface', 'kitchenType', 'landSurface',
       'hasLivingRoom', 'livingRoomSurface', 'hasBalcony', 'hasGarden',
       'gardenSurface', 'gardenOrientation', 'parkingCountIndoor',
       'parkingCountOutdoor', 'hasAirConditioning', 'hasArmoredDoor',
       'hasVisiophone', 'hasOffice', 'toiletCount', 'hasSwimmingPool',
       'hasFireplace', 'hasTerrace', 'terraceSurface', 'terraceOrientation',
       'accessibleDisabledPeople', 'epcScore', 'price'],
      dtype='object')

In [11]:
# Step 4: Drop unnecessary or highly null columns (example)
df = df.drop(columns=['Unnamed: 0', "url", "id", "monthlyCost", "accessibleDisabledPeople", "hasBalcony"], errors='ignore')


In [5]:
df.shape

(80368, 47)

In [6]:
# Step 5: Handle Missing Values 
df = df.dropna(subset=["price"])  # Drop if price is missing
#df = df.fillna(0)  # Simple fill for numeric NA
print(df.shape)



(76370, 47)


In [12]:
missing_facede_df = df[df['facedeCount'].isna()]
missing_counts_by_subtype = missing_facede_df['subtype'].value_counts()
print(missing_counts_by_subtype)

subtype
APARTMENT               11138
HOUSE                    7327
APARTMENT_GROUP          2425
HOUSE_GROUP              1565
FLAT_STUDIO               656
GROUND_FLOOR              643
VILLA                     614
DUPLEX                    571
PENTHOUSE                 463
APARTMENT_BLOCK           442
MIXED_USE_BUILDING        314
SERVICE_FLAT              255
KOT                       151
FARMHOUSE                  94
LOFT                       90
MANSION                    87
TOWN_HOUSE                 67
EXCEPTIONAL_PROPERTY       58
BUNGALOW                   57
COUNTRY_COTTAGE            33
TRIPLEX                    29
CHALET                     19
OTHER_PROPERTY             14
CASTLE                     12
MANOR_HOUSE                 5
Name: count, dtype: int64


In [ ]:
summary = df.groupby('subtype').agg(
    count=('subtype', 'count'),
    landSurface_missing=('landSurface', lambda x: x.isna().mean()),
    habitableSurface_missing=('habitableSurface', lambda x: x.isna().mean())
).sort_values(by='count', ascending=False)

# Convert to percentages
summary['landSurface_missing_%'] = (summary['landSurface_missing'] * 100).round(2)
summary['habitableSurface_missing_%'] = (summary['habitableSurface_missing'] * 100).round(2)

# Select relevant columns
summary = summary[['count', 'landSurface_missing_%', 'habitableSurface_missing_%']]

print(summary)

                      count  landSurface_missing_%  habitableSurface_missing_%
subtype                                                                       
HOUSE                 33971                  11.95                       13.71
APARTMENT             23844                 100.00                        7.76
VILLA                  3769                   4.27                        9.60
APARTMENT_BLOCK        2454                  31.01                       20.17
MIXED_USE_BUILDING     1840                  20.43                       16.79
GROUND_FLOOR           1771                 100.00                        4.80
DUPLEX                 1691                 100.00                        5.80
FLAT_STUDIO            1381                 100.00                       10.57
PENTHOUSE              1265                 100.00                        4.82
EXCEPTIONAL_PROPERTY    813                   9.59                       10.95
MANSION                 544                  12.50  

In [ ]:
# 1. Compute price per m² for houses
houses = df[(df['type'].str.upper() == 'HOUSE') & df['landSurface'].notna() & df['price'].notna()].copy()
houses['price_per_m2'] = houses['price'] / houses['landSurface']
house_postCode_price = houses.groupby('postCode')['price_per_m2'].mean().reset_index()
house_postCode_price.rename(columns={'price_per_m2': 'house_postCode_avg_price_per_m2'}, inplace=True)

# 2. Compute price per m² for apartments
apartments = df[(df['type'].str.upper() == 'APARTMENT') & df['price'].notna() & df['habitableSurface'].notna()].copy()
apartments['price_per_m2'] = apartments['price'] / apartments['habitableSurface']
apartment_postCode_price = apartments.groupby('postCode')['price_per_m2'].mean().reset_index()
apartment_postCode_price.rename(columns={'price_per_m2': 'apartment_postCode_avg_price_per_m2'}, inplace=True)

# 3. Merge both features into the full dataset
df = df.merge(house_postCode_price, on='postCode', how='left')
df = df.merge(apartment_postCode_price, on='postCode', how='left')

# 4. Set the correct values for house and apartment types
# Set apartment's house_avg_price_per_m2 to NaN
df.loc[df['type'].str.upper() == 'APARTMENT', 'house_postCode_avg_price_per_m2'] = None

# Set house's apartment_avg_price_per_m2 to NaN
df.loc[df['type'].str.upper() == 'HOUSE', 'apartment_postCode_avg_price_per_m2'] = None

In [ ]:
df.to_csv('new_dataset.csv', index=False)

In [ ]:
df.head(50)

,type,subtype,bedroomCount,bathroomCount,province,locality,postCode,habitableSurface,roomCount,hasAttic,...,toiletCount,hasSwimmingPool,hasFireplace,hasTerrace,terraceSurface,terraceOrientation,epcScore,price,house_postCode_avg_price_per_m2,apartment_postCode_avg_price_per_m2
0,APARTMENT,APARTMENT,2.0,1.0,Brussels,Etterbeek,1040,100.0,NaN,NaN,...,1.0,NaN,NaN,True,7.0,NaN,C,399000.0,NaN,4436.991869
1,HOUSE,HOUSE,4.0,2.0,Brussels,Etterbeek,1040,270.0,12.0,True,...,3.0,NaN,NaN,True,1.0,NaN,E,895000.0,6855.940954,NaN
2,APARTMENT,APARTMENT,2.0,1.0,Brussels,Etterbeek,1040,87.0,NaN,NaN,...,1.0,NaN,NaN,True,2.0,NaN,F,465000.0,NaN,4436.991869
3,APARTMENT,APARTMENT,2.0,2.0,Brussels,Bruxelles,1040,104.0,NaN,NaN,...,2.0,NaN,NaN,True,3.0,NaN,B,590000.0,NaN,4436.991869
4,APARTMENT,FLAT_STUDIO,1.0,1.0,Brussels,Etterbeek,1040,71.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,E,289000.0,NaN,4436.991869
5,APARTMENT,APARTMENT,2.0,1.0,Brussels,ETTERBEEK,1040,90.0,NaN,NaN,...,1.0,NaN,NaN,True,2.0,NaN,D,375000.0,NaN,4436.991869
6,APARTMENT,DUPLEX,3.0,2.0,Brussels,Etterbeek,1040,220.0,NaN,NaN,...,4.0,NaN,NaN,True,25.0,SOUTH_WEST,D,950000.0,NaN,4436.991869
7,APARTMENT,DUPLEX,3.0,2.0,Brussels,Etterbeek,1040,220.0,NaN,NaN,...,4.0,NaN,NaN,True,25.0,SOUTH_WEST,D,950000.0,NaN,4436.991869
8,APARTMENT,DUPLEX,2.0,1.0,Brussels,Etterbeek,1040,187.0,NaN,NaN,...,2.0,NaN,NaN,True,25.0,SOUTH_WEST,C,795000.0,NaN,4436.991869
9,APARTMENT,APARTMENT,1.0,1.0,Brussels,Etterbeek,1040,93.0,NaN,NaN,...,1.0,NaN,NaN,True,5.0,EAST,F,297000.0,NaN,4436.991869


In [ ]:
house_df = df[df['type'].str.lower() == 'house'].copy()
apartment_df = df[df['type'].str.lower() == 'apartment'].copy()